In [24]:
import pandas as pd 

import pickle 
import nltk as nl
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk import WordNetLemmatizer

In [17]:
nl.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\momor\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

We import the vocabulary and the inverted index 

In [76]:
with open( 'vocabulary.pkl', 'rb') as f:
        vocabulary= pickle.load(f)
        f.close()


In [77]:
with open( 'Inverted_Index.pkl', 'rb') as f:
        ID= pickle.load(f)
        f.close()

We import the dataset too we clean it 

In [50]:
df=pd.read_csv('dataset/Dataset_300Pages.tsv', sep='\t')

df.dropna(subset=['Plot'],inplace=True)

df.reset_index(drop=True,inplace=True)

We will also need these functions to run the query:  

In [78]:
def clean_info (string):
    # this command will split the string given in input in substrings by using 
    # the words given to RegexpTokenizer as argument
    

    
    # filter the punctuation
    tmp = nl.RegexpTokenizer(r"['\w-]+").tokenize(string)  
    
    #first we lower all the words otherwise words such as AND,IS,MY are not consider stopwords 
    tmp = [word.lower() for word in tmp]
    
     # filter the stopwords (e.g. 'the', 'my', 'an', ...)
    tmp = [word for word in tmp if not word in stopwords.words("english")]
    
    #we lemmatize all the words (e.g. 'dirn')
    lemma = WordNetLemmatizer()
    tmp = [lemma.lemmatize(word, pos = "v") for word in tmp]    # v for verbs
    tmp = [lemma.lemmatize(word, pos = "n") for word in tmp]    # n for nouns
    
    
    final = ' '.join(tmp)
    
    return final

In [79]:
# we will need first these two functions to implement the search function 


# this function takes as input a list of lists and gives back the index of the list that has minimus first element

def find_min_list (L):
    min_elem = L[0][0]
    count = 0
    index = 0
    for l in L[1:]: 
        if min_elem > l[0]:
            count = count + 1 
            index = count 
            min_elem = l[0]
        else: 
            count = count + 1 

    return index 
        

# this function takes as input a list of lists and gives back the list created from the intersection of the lists 
    
def intersect_list (L):
    results = []
    while all(len(l) > 0 for l in L):

        if all([L[0][0] == l[0] for l in L[1:]]):     
            results.append(L[0][0])
            L = [l[1:] for l in L]

        else : 
            min_index = find_min_list(L)
            L[min_index] = L[min_index][1:]

    return results 

This is the function that finds the documents containing the query 

In [87]:
def find_query(inverted_index, vocabulary):
    
    string = input() # asks the user a string of words to look up 
    
    string_cleaned = clean_info(string)
    list_words = string_cleaned.split(' ')
    
    
    # now translate the list of words in term_id )
    list_termID = []
    for word in list_words: 
        list_termID.append(vocabulary.get(word))    
        
   

    # retrieve the documents in the inverted index and collect them in a list 
    list_documents = []
    for term_id in list_termID:
        if term_id in inverted_index.keys():   # checking if the word we are looking for is in the inverted_index
            list_documents.append(inverted_index.get(term_id))
    
     
    # now intersect these lists (HERE IT'S FUNDAMENTAL TO SUPPOSE THAT THE DOCUMENTS ARE COLLECTED AS INCREASING 
    # SEQUENCES)
    results = intersect_list(list_documents)
    
    
    return results 

In [85]:
results = find_query(ID, vocabulary)

horse riding


In [35]:
# to print the plots with the newlines
pd.set_option('display.max_colwidth', None) 

In [82]:
# we still have to print the table without the newlines 

df.iloc[results][['bookTitle', 'Plot', 'Url']]

,bookTitle,Plot,Url
87,"Written in Stone: Evolution, the Fossil Record, and Our Place in Nature","“Switek seamlessly intertwines two types of evolution: one of life on earth and the other of paleontology itself.”—Discover Magazine““In delightful prose, [Switek] . . . superbly shows that ‘[i]f we can let go of our conceit,’ we will see the preciousness of life in all its forms.”—Publishers Weekly (starred review)“Highly instructive . . . a warm, intelligent yeoman’s guide to the progress of life.”—Kirkus Reviews“Magisterial . . . part historical account, part scientific detective story. Switek’s elegant prose and thoughtful scholarship will change the way you see life on our planet. This book marks the debut of an important new voice.”—Neil Shubin“Elegantly and engagingly crafted, Brian Switek’s narrative interweaves stories and characters not often encountered in books on paleontology—at once a unique, informative and entertaining read.”—Niles Eldredge“If you want to read one book to get up to speed on evolution, read Written in Stone. Brian Switek’s clear and compelling book is full of fascinating stories about how scientists have read the fossil record to trace the evolution of life on Earth.”—Ann Gibbons“[Switek's] accounts of dinosaurs, birds, whales, and our own primate ancestors are not just fascinating for their rich historical detail, but also for their up-to-date reporting on paleontology’s latest discoveries.”—Carl Zimmer""After reading this book, you will have a totally new context in which to interpret the evolutionary history of amphibians, mammals, whales, elephants, horses, and especially humans.”—Donald R. ProtheroSpectacular fossil finds make today's headlines; new technology unlocks secrets of skeletons unearthed a hundred years ago. Still, evolution is often poorly represented by the media and misunderstood by the public. A potent antidote to pseudoscience, Written in Stone is an engrossing history of evolutionary discovery for anyone who has marveled at the variety and richness of life.",https://www.goodreads.com/book/show/8198230-written-in-stone
178,True History of the Kelly Gang,"“I lost my own father at 12 yr. of age and know what it is to be raised on lies and silences my dear daughter you are presently too young to understand a word I write but this history is for you and will contain no single lie may I burn in Hell if I speak false.”In True History of the Kelly Gang, the legendary Ned Kelly speaks for himself, scribbling his narrative on errant scraps of paper in semiliterate but magically descriptive prose as he flees from the police. To his pursuers, Kelly is nothing but a monstrous criminal, a thief and a murderer. To his own people, the lowly class of ordinary Australians, the bushranger is a hero, defying the authority of the English to direct their lives. Indentured by his bootlegger mother to a famous horse thief (who was also her lover), Ned saw his first prison cell at 15 and by the age of 26 had become the most wanted man in the wild colony of Victoria, taking over whole towns and defying the law until he was finally captured and hanged. Here is a classic outlaw tale, made alive by the skill of a great novelist.\n\n",https://www.goodreads.com/book/show/110090.True_History_of_the_Kelly_Gang
407,Prisoner of the Inquisition,"Zarita, only daughter of the town magistrate, lives a life of wealth and privilege. Indulged by her parents, she is free to spend her days as she pleases, enjoying herself in the company of an eligible young nobleman, horse riding, or leisurely studying the arts.Saulo, son of a family reduced by circumstances to begging, witnesses his father wrongfully arrested and dealt with in the most horrifying way. Hauled off to be a slave at sea and pursued by pirates he encounters the ambitious mariner explorer, Christopher Columbus. Throughout his hardships Saulo is determined to survive - for he has sworn vengeance on the magistrate and his family.As Zarita's life also undergoes 